In [1]:
import random
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.externals.six import StringIO   
from sklearn.tree import export_graphviz
import pydotplus

# Q:預測大學生是否為夯哥/夯姐(Winner)

## Attribute
### 1. activitynum:參加的活動數，範圍:0-20
### 2. fbfriend:FB的好友數，範圍:0-1200    (在此處不影響結果)
### 3. igfan:IG的追蹤數，範圍:0-1200          (在此處不影響結果)
### 4. joinsa:是否有加入學生會，0:否；1:是
### 5. joinballteam:是否有加入球隊，0:否；1:是
### 6. humorous:是否幽默，0:否；1:是
### 7. gpa:學業成績，範圍:0-4.3
### 8. bodyvalue:身材分數，範圍:1-10
### 9. facevalue:顏值，範圍:1-10
### 10. popular:是否是夯哥/夯姐，0:否(Loser)；1:是(Winner)


In [2]:
df = pd.DataFrame(columns=['activitynum', 'fbfriend', 'igfan', 'joinsa', 'joinballteam',
                           'humorous', 'gpa', 'bodyvalue', 'facevalue', 'popular'])

In [3]:
for i in range(10000):
    tmp = pd.Series({'activitynum':random.randint(0,20), 'fbfriend':random.randint(0,1200), 
                     'igfan':random.randint(0,1200), 'joinsa':random.randint(0,1),
                     'joinballteam':random.randint(0,1),'humorous':random.randint(0,1), 
                     'gpa':random.uniform(0, 4.3), 'bodyvalue':random.randint(1,10), 
                     'facevalue':random.randint(1,10), 'popular':random.randint(0,1)})
    df = df.append(tmp, ignore_index=True)


# Decision tree based on my rule

## Rule:以活動參加數多寡分為三類
### 
### 1.第一類人(16-20)參加很多活動
#### 1-1若幽默>>Winner
#### 1-2不幽默但顏值高>>Winner
#### 1-3兩項特質都無，只是愛跑活動>>Loser
### 2.第二類人(9-15)參加活動也不少
#### 2-1若身材好，為球隊一分子且顏值高>>Winner
#### 2-2若身材好，為球隊一分子但顏值普通>>Loser
#### 2-3若只有身材好>>Loser
#### 2-4若身材不佳，但幽默且顏值高>>Winner
#### 2-5若身材不佳，但幽默且顏值普通>>Loser
#### 2-6若身材不佳又不幽默>>Loser
### 3.第二類人(0-8)參加少數活動或不參加
#### 3-1若身材好，為球隊一分子且顏值高>>Winner
#### 3-2若身材好，為球隊一分子但顏值普通>>Loser
#### 3-3若只有身材好>>Loser
#### 3-4若身材不佳，但顏值高>>Winner
#### 3-5若身材不佳，顏值普通但是是學霸>>Winner
#### 3-6若身材不佳，顏值、成績普通，但加入學生會>>Winner
#### 3-7若身材不佳，顏值、成績普通，也沒加入學生會>>Loser
#### 3-8若身材不佳，顏值普通且成績不佳>>Loser
#### 3-9若身材不佳，顏值不佳>>Loser
![](https://i.imgur.com/mCMhNNK.png)

In [4]:
for i in range(df.shape[0]):
    if df.loc[i,'activitynum'] > 15:
        if df.loc[i,'humorous'] == 1:
            df.at[i,'popular'] = 1
        else:
            if df.loc[i,'facevalue'] > 7:
                df.at[i,'popular'] = 1
            else:
                df.at[i,'popular'] = 0
    elif df.loc[i,'activitynum'] > 8:
        if df.loc[i,'bodyvalue'] > 5:
            if df.loc[i,'joinballteam'] == 1:
                if df.loc[i,'facevalue'] > 7:
                    df.at[i,'popular'] = 1
                else:
                    df.at[i,'popular'] = 0
            else:
                df.at[i,'popular'] = 0
        else:
            if df.loc[i,'humorous'] == 1:
                if df.loc[i,'facevalue'] > 7:
                    df.at[i,'popular'] = 1
                else:
                    df.at[i,'popular'] = 0
            else:
                df.at[i,'popular'] = 0
    else:
        if df.loc[i,'bodyvalue'] > 5:
            if df.loc[i,'joinballteam'] == 1:
                if df.loc[i,'facevalue'] > 7:
                    df.at[i,'popular'] = 1
                else:
                    df.at[i,'popular'] = 0
            else:
                df.at[i,'popular'] = 0
        else:
            if df.loc[i,'facevalue'] > 7:
                df.at[i,'popular'] = 1
            elif df.loc[i,'facevalue'] > 3:
                if df.loc[i,'gpa'] > 4:
                    df.at[i,'popular'] = 1
                elif df.loc[i,'gpa'] > 3:
                    if df.loc[i,'joinsa'] == 1:
                        df.at[i,'popular'] = 1
                    else:
                        df.at[i,'popular'] = 0
                else:
                    df.at[i,'popular'] = 0
            else:
                df.at[i,'popular'] = 0

In [5]:
df_train = df[:8000]
df_train.head()
df_test = df[8000:]
df_test.head()

y = df_train['popular'].values
df_train = df_train.drop('popular', 1)

In [6]:
dtree=DecisionTreeClassifier(max_depth=4)
dtree.fit(df_train,y)

dot_data = StringIO()
export_graphviz(dtree, 
                out_file=dot_data,  
                filled=True, 
                feature_names=list(df_train),
                class_names=['Loser','Winner'],
                special_characters=True)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_pdf("tree.pdf")

True

In [7]:
dtree.feature_importances_

array([0.21346011, 0.        , 0.        , 0.        , 0.05691159,
       0.24113546, 0.00474703, 0.08275822, 0.40098759])

In [8]:
y_test = df_test['popular'].values
X_test = df_test.drop('popular', 1)

y_predict = dtree.predict(X_test)

y_predict

array([0., 0., 0., ..., 1., 1., 0.])

# Accuracy

In [9]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_predict)

0.9595

# Decision tree based on sklearn.tree
![](https://i.imgur.com/axFjtEi.jpg)

# Discussion

### 1.FB好友(FBFRIEND)以及IG追蹤數(IGFAN)這兩個atrtribute在rule裡面並沒有出現，不會對一個人是否為夯哥(姐)造成影響，在Decision Tree裡面也沒有將此二項作為判斷的node。
### 2.基本上只要有高顏值(FACEVALUE)就會成為夯哥(姐)，是主要的判斷依據，而此屬性也出現在Decision Tree的root node，作為基本二分的首要條件。
### 3.若有好的身材(BODYVALUE)、良好運動能力(JOINBALLTEAM)以及幽默感(HUMOROUS)，也可以成為夯哥(姐)，這些屬性也出現在Decision Tree的root node，其中身材(BODYVALUE)要搭配良好運動能力(JOINBALLTEAM)才有成為夯哥(姐)的機會，可由左半部及右半部各有一個身材(BODYVALUE)看出來兩者需要同時出現才能產生成為夯哥(姐)的結果。
### 4.參加活動數本來在我自己的rule內是最一開始的分類依據，但並不是最後影響結果最重要的屬性，在Decision Tree第二層出現；而右下半部出現的另一個參加活動數的node，其中它的gini值為同層最高，可見得不是一個最好的分類條件。
### 5.若前面所提的屬性數值都不高，在我的rule內仍有設置能成為夯哥(姐)的條件:學習成績(GPA)和加入學生會(JOINSA)，但要同時滿足，且前者的條件高情況下，若判斷到此項基本上不太可能成為夯哥(姐)；而後者在Decision Tree中並沒有出現，判斷原因為:因為實際上要和高學習成績(GPA)搭配，在沒有其他搭配屬性情況下，對於決定分支結果沒有太重大的影響。